# OTSU Big Data Cloud BE

Welcome to this training of ISAE Supaero OTSU Cloud, Big Data and Machine Learning module. 

The end goal of this exercise is to build temporal statistics from several Seninel-2 L2A products accessed from an object storage. In order to do this, we'll read the products using (rio)Xarray with a Dask backend, directly from Google Cloud Storage, compute a NDSI over ten dates and plot snow cover evolution accross the period.

By doing this, we will learn the folowing things:

- How to access a Cloud storage and browse its objects,
- How to use rioxarray to access and load Satellite imagery,
- How to use Xarray Dataset class to build a complete timeseries dataset,
- How to chunk our data and use Dask to perform bigger than memory or distributed analysis over our entire dataset.

## Imports and settings

As always, this begins with imports. We sen set some environment variable to easily acces Google Cloud Storage anonymously.

In [ ]:
import gcsfs
import rioxarray
import rasterio
import os
import numpy as np
from distributed import Client
import xarray as xr
import pandas as pd

In [ ]:
def set_env():
    os.environ["GS_NO_SIGN_REQUEST"] = "YES"

set_env()

## Check you have access to GCS bucket

We can use gcsfs package, which mimics a file system usage on an object storage.

It is pretty easy on a public bucket with anonymous access authorized, like the one we use. You can see that an object store with fsspec can behave a lot like a standard fil system. Just don't forget it is not.

In [ ]:
import gcsfs
fs = gcsfs.GCSFileSystem(bucket_name="supaero", token='anon')

In [ ]:
fs.ls('supaero/31TCH')

In [ ]:
fs.ls('supaero/31TCH/SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2')

### The products

As you can see, we'll use Sentinel-2 L2A products, which have been create using Maja algorithm from CNES.
Do not hesitate to check Maja or [Theia website](https://www.theia-land.fr/en/product/sentinel-2-surface-reflectance/) to know more about these.

For example :

- How many bands a product has?
- What correction to the L1C product does MAJA makes?
- Have every band the same resolution (this is important for the following)?

## Reading a product band using rioxarray and rasterio

Rasterio is the Pythonic interface to GDAL, the go to tool when dealing with satelite products. Rioxarray is a library that makes it easy to read rasterio compatible products as Xarray DataArray or Datasets.

At first, we'll read only a subsection of a product, using classical Numpy slices. This way, only selected pixels will be loaded into memory (important when using Dask afterwards, and to not blow up your computer or server memory).

Just use the rioxarray.open_rasterio function to open a random tif file from above, then slice it in order to keep x and y dimension from pixels 4000 to 5000.
Wee'll have use an URL like "gs://bucket/path".

Opening the file should be pretty fast. Then plot the dataset in another cell, data should really be accessed at this point.

In [ ]:
%%time
swir = rioxarray.open_rasterio("gs://supaero/31TCH/SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2/SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2_FRE_B11.tif")
swir = swir[:,4000:5000,4000:5000]
swir

In [ ]:
%%time
swir.plot()

Even with the swir band, we can see we are in mountainous area.

## Building a single date Dataset

First, without using Dask, will build an Xarray Dataset from two DataArray that share the same dimensions and coordinates.

As we'll read two bands that to not share the same resolution, we'll have to resample the green band (using a simple mean).

Then, we'll use the Dataset to compute two more variable, NDSI, and snow mask.

### Build DataArrays

So first, we'll read the swir band as from above, but this time, we'll need to make sure that we handle the nodata value correctly (this will be important in the next part of this exercise). To do so, as these L2A products are note really standard, we need to manually remove the no data values using a filter. Then write it in rioxarray metadata.

So open the swir band as above, slice it the same way, but then user xarray.where to remove nodata values (-10_000), and use rioxarray write_no_data with encoded and inplace kwargs to set it correctly inside the data array.

In [ ]:
swir = rioxarray.open_rasterio("gs://supaero/31TCH/SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2/SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2_FRE_B11.tif")
swir = swir[:,4000:5000,4000:5000]
#No data handling
swir = swir.where(swir != -10000)
swir.rio.write_nodata(-10000, encoded=True, inplace=True)
swir

In [ ]:
swir.plot()

Note that Xarray with numpy backend is already a bit lazy:
- If you only open a file, only its metadata are read, nothin is loaded until an operation like plot or load is performed.
- When applying a metadata filter, we transform and load the data, how can you see it in the HTML representation of the DataArray in the notebook?

Now, we need to open the green band. Note that its resolution is two times better on spatial dimension than the SWIR band.

In order to build a dataset or perform operations, we need to resample the data to the same resolution as the SWIR band.

There are several ways to do this, including reproject from rioxarray, but since here we only want to divide resolution by exactly two, we'll use a simple mean through the coarsen function of Xarray.

So to sum up, in the following cell you'll need to:
- Open the green band of the same product as above,
- Slice the data before reading it, we only want to load only the data of interest. Be carefull, we've got twice as many pixels at the beginning, so be sure to take the correct amount of pixels, and at the correct indices.
- Apply coarsen to the result, with x=2, y=2, boundary='pad', and take the mean.
- As above, filter nodata values.

In [ ]:
green = rioxarray.open_rasterio("gs://supaero/31TCH/SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2/SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2_FRE_B3.tif")
green = green[:,8000:10000,8000:10000]
# Rééchantillonage à 20m, diviser résolution par 2
green = green.coarsen(x=2, y=2, boundary='pad').mean()
#No data
green = green.where(green != -10000)
green.rio.write_nodata(-10000, encoded=True, inplace=True)
green

Then plot the DataArray you just created.

You should observe the same mountains, but this time, with the green band, you should see some snow!

In [ ]:
green.plot()

### Compute the NDSI using DataArrays

Since the two DataArrays are of the same dimension, you can already perform operations with both of them.

Use them two compute the [Normalised Difference Snow Index (NDSI)](https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/ndsi/).

This should be as straigthforward as with Numpy.

In [ ]:
ndsi = (green - swir) / (green + swir)
ndsi

Did you tried to plot it?

In [ ]:
ndsi.plot()

It has been established by research that a level of 0.4 in NDSI usually means there is snow.

Just use this threshold to compute a boolean DataArray representing the snow cover over this particular area, and plot it. There should be plenty of snow on your image.

In [ ]:
(ndsi > 0.4).plot()

### Xarray Datasets

Now, we'll just use what we've done above to build a Simple Dataset made of several variables, because this will be handy in the next part of the exercise.

Building a Dataset is as simple as using its constructor (xr.Dataset()) and giving it a dict with a string name associated to a DataArray.

Build a dataset with a green and swir variable pointing to the DataArray we built above, then display it to see its structure. You can also play with the Database buttons or metadata to have more informations.

In [ ]:
sub_ds = xr.Dataset({"green": green, "swir": swir})
sub_ds

A nice feature about Dataset is to be able to build and store new variables. We'll see also in the next part that we can add Temporal index as dimensions.

For now, we'll just create two new variable in this Dataset:
- a ndsi variable, that is linked to the NDSI computation over the two other variables,
- a snow variable, boolean array as above.

Adding variable is a simple as affecting a new Column in pandas, e.g. dataset["ndsi"] = ...

In [ ]:
sub_ds["ndsi"] = (sub_ds.green - sub_ds.swir) / (sub_ds.green + sub_ds.swir)
sub_ds["snow"] = sub_ds.ndsi > 0.4
sub_ds

Then plot the snow variable, just to check:

In [ ]:
sub_ds.snow.plot()

### Compute snow percentage over the area

The end goalg below is to plot the evolution of snow cover accross a time serie. But how to plot a snow cover percentage over one product?

You'll need to count pixels having snow, and divide this number by the number of pixels wich are not nodata.

In [ ]:
sub_ds.snow.sum() / sub_ds.snow.count()

On this particular patch, you should get over 77% of snow cover.

## Compute a time series analysis

Now that we know how to create a single timestamp dataset, we'll build a dataset with a time dimension.
The idea is to stack single temporal datasets into a single one using a new time dimension.

Up to now, we've only built datasets that easily fit in memory, by taking only part of one observation. 
In order to be able to work on full images and on ten products, we'll need to use Dask.

### Start a Dask cluster

Configure it according to your computing power. Then we'll need to set environment variable on every worker.

In [ ]:
client = Client(n_workers=2, threads_per_worker=2, memory_limit='1GiB')
client

In [ ]:
client.run(set_env)

It's really interesting when using Dask to start a real Distributed cluster (even on one machine like we've done above).

This gives access to a nice Dashboard, just click on the link displayed above (ending with 8787/status). If you are using binder, you should replace the URL with something like:

https://hub.2i2c.mybinder.org/user/guillaumeeb-supaero-otsu-course-4kfqus3j/proxy/8787/status

First part of the URL should be copied from you Jupyterlab URL on your browser.

### Define some functions to build Datasets

OK, in order to simplify the building of our time serie, we'll define some functions for the following:
- Reading one band of a product. It should also handle an optional resampling. This time, we also want to use a Dask backend.
- Creating a single time Dataset with green and swir band.

So first, we'll define a read_one_band(product, band, coarsen=1) function:
- product is the name of the product, i.e. the "SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2" part, which is repeated in a folder, and in the file name.
- band is the band identifier, i.e. B3 or B11 in our case. We always access "FRE" bands in our case.
- coarsen is the level of subsambling, it should be 1 by default (no subsampling) for the SWIR band, set it to 2 for GREEN to perform a resampling.
- When opening the file with rioxarray, we'll use two new arguments: chunks, and lock=False. chunks indicate that we want chunked array as a backend, so Dask Arrays. You use (-1, 1024 * coarsen, 1024*coarsen) as a value before opening the file.
- We also want to remove the band dimension that is useless, in order to do so, just apply .squeeze('band', drop=True) right after the opening.

In [ ]:
def read_one_band(product, band, coarsen=1):
    chunks=(-1, 1024*coarsen, 1024*coarsen)
    band = rioxarray.open_rasterio(f"gs://supaero/31TCH/{product}/{product}_FRE_{band}.tif", 
                                chunks=chunks,
                                lock=False).squeeze('band', drop=True)
    band = band.where(band != -10000)
    band.rio.write_nodata(-10000, encoded=True, inplace=True)
    if coarsen > 1:
        band = band.coarsen(x=coarsen, y=coarsen, boundary='pad').mean()
    return band

Try this function on the green band of "SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2", so with a coarsen=2 value, and whatch the HTML repr. What do you notice?

In [ ]:
read_one_band("SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2", "B3", 2)

Now, we'll define a create_dataset(product) function.

Product argument is the same as above, and it will create a 2 dimension dataset with two DataArray variables: green and swir, at the same resolution.

In [ ]:
def create_dataset(product):
    ds = xr.Dataset({"green": read_one_band(product, "B3", 2), "swir": read_one_band(product, "B11")})
    return ds

Try it on the above product

In [ ]:
create_dataset("SENTINEL2B_20191224-104910-788_L2A_T31TCH_C_V2-2")

### Build our time serie

We are now ready to build a time series of our ten products.

In order to do that, we'll need:
- our product list (which is built in the below cell)
- A list of containing all of our datasets
- A Pandas Datetime index corresponding to our product list (code also given)
- Apply xarray concat on our dataset list associated with our new Index as a dimension.

In [ ]:
product_list = [path.split("/")[-1] for path in fs.ls('supaero/31TCH')]
product_list

In [ ]:
# Create time index
dates = [product.split("_")[1] for product in product_list]
dt_index = pd.to_datetime(dates, format="%Y%m%d-%H%M%S-%f")
dt_index.name = "time"
dt_index

So now, create a list by creating a dataset from all of the products above.

Then, use xr.concat to create a new overall dataset with a new time dimension and all of our single time step datasets in it.

In [ ]:
datasets = []
for product in product_list:
    datasets.append(create_dataset(product))

In [ ]:
complete_ds = xr.concat(datasets, dt_index)
complete_ds

### Add NDSI and snow mask

Just as with a single time step Dataset, you can easily add two new variables to this complete Dataset as above.

So create the new ndsi and snow variables, as above.

In [ ]:
complete_ds["ndsi"] = (complete_ds.green - complete_ds.swir) / (complete_ds.green + complete_ds.swir)
complete_ds["snow"] = complete_ds.ndsi > 0.4

### Compute snow percentage time series over the whole image

Did you notice?

All the operation above where pretty fast, no? That's because we didn't really computed anything. By using Dask backend and Xarray, we only worked using product metadata, all the real values are still waiting to be computed. This is the lazy power of Dask with Xarray.

But now, if we want some result, we'll need to trigger a computation.

What we are interested in is a time series representing the snow percentage evolution over the ten dates.

We'll need to count snow pixel and sum them over spatial dimension, count also the not nodata values over each time step on the spatial dimension, and just divide the two time series to get the percentage.

In order to trigger the computation, we'll need to use compute method.

Then whatch the Daks Dashboard. This should take some time, depending on your resources (on Binder it does).

In [ ]:
%%time
snow_percentage = (complete_ds.snow.sum(dim=["x", "y"]) / complete_ds.ndsi.count(dim=["x", "y"])).compute()

In [ ]:
snow_percentage

### And what is the result?

Now, we just want to plot the results.

As we've not sorted our dataset by dates at the beginning, you'll want to do it first, using xarray sortby method.

Then just plot, or better, plot.step(where="mid") for a nicer view (in my opinion).

Does this snow cover percentage time serie looks right?

In [ ]:
snow_percentage.sortby("time").plot()

In [ ]:
snow_percentage.sortby("time").plot.step(where="mid")

# Extends other statistics

- Snow area
- NDSI
- Add elevation, and compute snowline